In [36]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import BertTokenizer, BertForSequenceClassification, AdamW

## Outage Detection

In [37]:
# Load and Preprocess the Data:
import pandas as pd

data = pd.read_csv('data.csv')
# Preprocess the data (e.g., remove unnecessary columns, clean text)
# data  = data[:200]

In [38]:
data[["outage"]].value_counts()

outage
1         20431
0         17638
dtype: int64

In [30]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from transformers import pipeline

# Load the unlabelled tweet data
data = pd.read_csv('data.csv')

# Data Preprocessing
# Perform any necessary data cleaning and preprocessing steps here

# Convert the 'text' column to string explicitly
data['text'] = data['text'].astype(str)

# Extract the text data
texts = data['text'].tolist()

# Use K-means clustering to generate labels for outage detection
num_clusters = 2  # Number of clusters to generate
kmeans = KMeans(n_clusters=num_clusters, random_state=42)

# Vectorize the text data using TF-IDF and fit K-means
vectorizer = TfidfVectorizer()
text_vectors = vectorizer.fit_transform(texts)
cluster_labels = kmeans.fit_predict(text_vectors)

# Assign labels to the data based on cluster labels
data['label'] = np.where(cluster_labels == 0, 'complaints', np.where(cluster_labels == 1, 'informational', 'appreciation'))

# Split the data into train and test sets
train_data, test_data, train_labels, test_labels = train_test_split(
    data['text'], data['label'], test_size=0.2, random_state=42
)

# Fine-tune a pre-trained transformer-based model for outage detection
classifier = pipeline('text-classification', model='bert-base-uncased')

# Set the batch size
batch_size = 32

# Predict on the test set in batches
test_predicted_labels = []
num_batches = len(test_data) // batch_size

for i in range(num_batches + 1):
    start_idx = i * batch_size
    end_idx = min((i + 1) * batch_size, len(test_data))
    batch_predictions = classifier(test_data[start_idx:end_idx].tolist())
    batch_predicted_labels = [label['label'] for label in batch_predictions]
    test_predicted_labels.extend(batch_predicted_labels)

# Calculate F1 score
f1 = f1_score(test_labels, test_predicted_labels, average='weighted')

# Generate classification report
report = classification_report(test_labels, test_predicted_labels)

# Generate confusion matrix
confusion_mat = confusion_matrix(test_labels, test_predicted_labels)

# Output the evaluation metrics
print('F1 Score: ', f1)
print('Classification Report:\n', report)
print('Confusion Matrix:\n', confusion_mat)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

F1 Score: 0.0
Classification Report:
                precision    recall  f1-score   support

      LABEL_0       0.00      0.00      0.00       0.0
      LABEL_1       0.00      0.00      0.00       0.0
   complaints       0.00      0.00      0.00    6755.0
informational       0.00      0.00      0.00     890.0

     accuracy                           0.00    7645.0
    macro avg       0.00      0.00      0.00    7645.0
 weighted avg       0.00      0.00      0.00    7645.0

Confusion Matrix:
 [[   0    0    0    0]
 [   0    0    0    0]
 [1082 5673    0    0]
 [   9  881    0    0]]


/Users/oowola01/opt/anaconda3/envs/mlwork/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/oowola01/opt/anaconda3/envs/mlwork/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/oowola01/opt/anaconda3/envs/mlwork/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_st

## Outage Characterization

In [ ]:
# if an outage is found from teh firest tep we can characterize it 
characteristics = ["storm", "trees", "animals", "equipment"]